# CH5.2 소규모 데이터셋에서 밑바닥부터 convnet 훈련하기

소규모 data를 사용해 img classification model을 만드는 일은 흔한 경우다

2000개의 train sample에서 ConvNet에 어떤 규제 방법도 사용하지 않고 train 할 경우 : **과대적합**으로 다소 낮은 71%의 acc를 달성

**overfitting**을 줄이기 위한 방법
1. data augmentation (이번 ch에서 다룸)
2. pretrained network 사용해 feature 추출
3. pretrained model tunning


## 1) 작은 dataset 문제에서 딥러니의 타당성

**deep learning의 근본적인 특징:**
1. **feature 공학의 수작업 없이 feature을 추출할 수 있다는 점** -> train sample이 많아야 함
    - sample의 수의 의미는 훈련하려는 network의 크기와 깊이에 상대적이다
    - model이 작고 규제가 잘 돼있으며 간단한 작업이라면, 수백개의 sample만으로도 풍분하다
    - **convnet은 지역적이고 평행이동으로 변하지 않는 feature을 학습하기 때문에 인식에 관한 문제에서 매우 효율적으로 data 사용**
        * 매우 작은 img dataset에서 어떤 종류의 feature 공학을 사용하지 않고도 convnet을 활용해 좋은 결과를 얻을 수 있다
        
        
2. **deep learning model은 매우 다목적이다**
    - 대규모 dataset에서 훈련시킨 img classification model을 조금만 변경시키면 완전히 다른 문제에도 사용 가능
    - 특히 computer vision에서는 ImageNet의 대규모 dataset에서 사전 훈련된 model을 공유하므로 매우 data가 적어도 강력한 vision model을 만들어낼 수 있다 

## 2) data 내려받기

data 구성 : 2000개의 img(1000개의 고양이, 1000개의 강아지)
- train data : 1000개( 500개 고양이, 500개 강아지)
- validation data : 500개(250개 고양이, 250개 강아지)
- test data : 500개(250개 고양이, 250개 강아지)    

In [1]:
import os, shutil

In [3]:

# 원본 데이터셋을 압축 해제한 디렉터리 경로
original_dataset_dir = './006975/datasets/cats_and_dogs/train'

# 소규모 데이터셋을 저장할 디렉터리
base_dir = './006975/datasets/cats_and_dogs_small'
if os.path.exists(base_dir):  # 반복적인 실행을 위해 디렉토리를 삭제합니다.
    shutil.rmtree(base_dir)   # 이 코드는 책에 포함되어 있지 않습니다.
os.mkdir(base_dir)

# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# 검증용 고양이 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# 처음 1,000개의 고양이 이미지를 train_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# 다음 500개 고양이 이미지를 validation_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 고양이 이미지를 test_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 처음 1,000개의 강아지 이미지를 train_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 validation_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 test_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [4]:
print('훈련용 고양이 이미지 전체 개수:', len(os.listdir(train_cats_dir)))

훈련용 고양이 이미지 전체 개수: 1000


In [5]:
print('훈련용 강아지 이미지 전체 개수:', len(os.listdir(train_dogs_dir)))

훈련용 강아지 이미지 전체 개수: 1000


In [6]:
print('검증용 고양이 이미지 전체 개수:', len(os.listdir(validation_cats_dir)))

검증용 고양이 이미지 전체 개수: 500


In [7]:
print('검증용 강아지 이미지 전체 개수:', len(os.listdir(validation_dogs_dir)))

검증용 강아지 이미지 전체 개수: 500


In [8]:
print('테스트용 고양이 이미지 전체 개수:', len(os.listdir(test_cats_dir)))

테스트용 고양이 이미지 전체 개수: 500


In [9]:
print('테스트용 강아지 이미지 전체 개수:', len(os.listdir(test_dogs_dir)))

테스트용 강아지 이미지 전체 개수: 500


## 3) network 구성하기

Conv2D(relu activation)+ MaxPooling2D layer을 번갈아 쌓음
* 복잡한 문제이기 때문에 network를 좀더 크게 만들 것임
* flatten layer의 크기가 너무 커지지 않도록 feature map의 크기를 줄일 수 있음
* binary classification 문제이므로 network는 하나의 unit(크기가 1인 dense 층)과 sigmoid activation으로 마무리
    - sigmoid unit은 한 class에 대한 확률을 encoding한다

> **NOTE**    
> feature map의 깊이는 network에서 점진적으로 증가하지만, feature map의 크기는 감소한다
>* 깊이: 32에서 128까지
>* 크기: 150x150에서 7x7까지

In [10]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

* 하나의 sigmoide unit으로 확률이 출력되므로 loss를 binary_crossentropy 사용

In [12]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

## 4) data 전처리

**data는 network에 주입되기 전에 부동소수 타입의 tensor로 전처리되어야 함**   
차례:
1. JPEG 사진 파일을 읽는다
2. JPEG를 RGB pixel 값으로 decoding한다
3. 그 다음 부동소수 타입(float)의 tensor로 변환
4. pixel 값의 scale을 (0, 255) 사이에서 [0,1] 사이로 조정 : normaliztion
    - 신경망은 작은 input값을 선호

>`keras.preprocessing.image`   
> 위 라이브러리에 img 처리를 위한 helper 도구들이 있다
> * 특히 `ImageDataGenerator` class는 img 파일을 전처리 배치 tensor로 자동으로 바꿔주는 **generator**를 만들어줌

In [13]:
from keras.preprocessing.image import ImageDataGenerator

# 모든 이미지를 1/255로 스케일을 조정합니다
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 150 × 150 크기로 바꿉니다
        target_size=(150, 150),
        batch_size=20,
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [14]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (20, 150, 150, 3)
배치 레이블 크기: (20,)


In [15]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

C:\Users\bomsi\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
100/100 [==============================] - 28s 274ms/step - loss: 0.6974 - acc: 0.5057 - val_loss: 0.6766 - val_acc: 0.5750
Epoch 2/30
100/100 [==============================] - 26s 261ms/step - loss: 0.6698 - acc: 0.5820 - val_loss: 0.7153 - val_acc: 0.5020
Epoch 3/30
100/100 [==============================] - 27s 265ms/step - loss: 0.6371 - acc: 0.6327 - val_loss: 0.6352 - val_acc: 0.6300
Epoch 4/30
100/100 [==============================] - 27s 265ms/step - loss: 0.5831 - acc: 0.7105 - val_loss: 0.5952 - val_acc: 0.6890
Epoch 5/30
100/100 [==============================] - 27s 266ms/step - loss: 0.5473 - acc: 0.7395 - val_loss: 0.6818 - val_acc: 0.5980
Epoch 6/30
100/100 [==============================] - 28s 275ms/step - loss: 0.5386 - acc: 0.7223 - val_loss: 0.6868 - val_acc: 0.6400
Epoch 7/30
100/100 [==============================] - 33s 329ms/step - loss: 0.4881 - acc: 0.7628 - val_loss: 0.5645 - val_acc: 0.7060
Epoch 8/30
100/100 [==============================] - 2

## 5) data 증식 사용하기